In [1]:
import cv2
import numpy as np
import tensorflow as tf
import time
import os
import matplotlib.pyplot as plt

In [2]:
H = 256
W = 256


prev_frame_time = 0
new_frame_time = 0

In [3]:
model = tf.keras.models.load_model("test_mdl/resnet_unet_convlstm_aug_val.h5")

In [4]:
imgBg = cv2.imread("bg/1.jpg")
listImg = os.listdir("bg")
indexImg = 0

imgList = []
for imgPath in listImg:
    img = cv2.imread(f'bg/{imgPath}')
    imgList.append(img)

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    h, w, _ = frame.shape
    ori_frame = frame

    frame = cv2.resize(frame, (W, H))
    frame = np.expand_dims(frame, axis=0)
    frame = np.expand_dims(frame, axis=0)
    frame = frame / 255.0

    mask = model.predict(frame)[0]
    mask = np.squeeze(mask, axis=0)
    mask = cv2.resize(mask, (w, h))
    mask = mask > 0.5
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    
    photo_mask = mask
    background_mask = np.abs(1 - mask)

    masked_frame = ori_frame * photo_mask

    background_mask = np.concatenate([background_mask, background_mask, background_mask], axis=-1)
    background_mask = background_mask * imgList[indexImg]
    final_frame = masked_frame + background_mask
    final_frame = final_frame.astype(np.uint8)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    fps = int(fps)
    fps = str(fps)
    
    cv2.putText(final_frame, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)
    
    cv2.imshow('Bg replacement', final_frame)

    # Break loop outcome
    key = cv2.waitKey(1)
    if key == ord('a'):
        if indexImg > 0:
            indexImg -= 1
    elif key == ord('d'):
        if indexImg < len(imgList) - 1:
            indexImg += 1
    elif key == ord('q'):
        break
        
# Releases webcam or caputer device
cap.release()
# Closes imshow frame
cv2.destroyAllWindows()

1/1 [==============================] - 0s 56ms/step


In [ ]:
# Releases webcam or caputer device
cap.release()
# Closes imshow frame
cv2.destroyAllWindows()